In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
np.random.seed(1)

import sys
import sklearn
import sklearn.ensemble
import sklearn.cluster
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

import shap
from anchor import utils
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality', discretize=False, balance=True, dataset_folder=dataset_folder)

In [3]:
dataset.train[:,12] = 1
msk = np.where(dataset.labels_train == 1)[0]
msk = msk[:int(msk.shape[0]*0.85)]
dataset.train[msk,12] = 0

dataset.test[:,12] = 1
msk = np.where(dataset.labels_test == 1)[0]
print(msk.shape)
mskk = msk[:int(msk.shape[0]*0.85)]
print(mskk.shape)
dataset.test[mskk,12] = 0

In [4]:
X_train = pd.DataFrame(data=dataset.train, columns=dataset.feature_names)
y_train = dataset.labels_train
X_test = pd.DataFrame(data=dataset.test, columns=dataset.feature_names)
y_test = dataset.labels_test

In [5]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=70, n_jobs=5, random_state=1)
c.fit(X_train, y_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(X_train)))
print('Test', sklearn.metrics.accuracy_score(y_test, predict_fn(X_test)))

Train 1.0
Test 1.0


In [6]:
explainer = shap.KernelExplainer(predict_fn, X_train.median())

In [7]:
exp = explainer.shap_values(X_test)

100%|████████████████████████████████████████████████████████████████████████████████| 836/836 [03:10<00:00,  4.22it/s]


In [8]:
import copy
test_ori = []
enc_feats = []
sz = dataset.test.shape
for i in range(sz[0]):
    temp = list(copy.deepcopy(dataset.test[i]))
    for j in range(sz[0]):
        if j in dataset.categorical_features:
            temp[j] = dataset.categorical_names[j][int(temp[j])].replace(',','')
    test_ori.append(temp)

ori_data = pd.DataFrame(test_ori, columns=dataset.feature_names)

feats = copy.deepcopy(dataset.feature_names)

In [9]:
num_feats = len(dataset.feature_names)
sz = exp.shape
imp = np.zeros(num_feats)
for i in range(sz[0]):
    for j in range(sz[1]):
        imp[j] = imp[j] + abs(exp[i][j])
    

In [10]:
order = np.argsort(-imp)
a = list(np.array(feats)[order])
a

['smoke',
 'age',
 'racewb',
 'sex',
 'mets_achieved',
 'resting_systolic',
 'percent_hr_achieved',
 'reason',
 'diuretic',
 'htnmed',
 'htn',
 'dm',
 'famhx',
 'afib2']

In [11]:
preds =  predict_fn(dataset.test)

In [12]:
view1 = ori_data.reindex(columns=a)
view1.insert(loc=0,column='dead',value=dataset.labels_test)
view1.insert(loc=1,column='prediction',value=preds)
yesmap={0:'No', 1:'Yes'}
yes_no = lambda x: yesmap[x] 
view1['dead']=view1['dead'].apply(yes_no)
view1['prediction']=view1['prediction'].apply(yes_no)

In [ ]:
view1

In [15]:
view1.to_csv('bmortshap.csv',index=False)